<a href="https://colab.research.google.com/github/Saravanan-SD/Basic-Python/blob/main/04_Transfer_learning_Feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref= zipfile.ZipFile("10_food_classes_10_percent.zip")

zip_ref.extractall()

zip_ref.close()


--2023-06-12 10:18:01--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 172.217.218.128, 142.250.153.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M  39.0MB/s    in 4.7s    

2023-06-12 10:18:06 (34.2 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [ ]:
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directory and {len(filenames)} images in {dirpath}")

There are 2 directory and 0 images in 10_food_classes_10_percent
There are 10 directory and 0 images in 10_food_classes_10_percent/test
There are 0 directory and 250 images in 10_food_classes_10_percent/test/hamburger
There are 0 directory and 250 images in 10_food_classes_10_percent/test/ramen
There are 0 directory and 250 images in 10_food_classes_10_percent/test/sushi
There are 0 directory and 250 images in 10_food_classes_10_percent/test/ice_cream
There are 0 directory and 250 images in 10_food_classes_10_percent/test/steak
There are 0 directory and 250 images in 10_food_classes_10_percent/test/chicken_curry
There are 0 directory and 250 images in 10_food_classes_10_percent/test/fried_rice
There are 0 directory and 250 images in 10_food_classes_10_percent/test/grilled_salmon
There are 0 directory and 250 images in 10_food_classes_10_percent/test/pizza
There are 0 directory and 250 images in 10_food_classes_10_percent/test/chicken_wings
There are 10 directory and 0 images in 10_food

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE= (224,224)
BATCH_SIZE=32

train_dir="10_food_classes_10_percent/train/"

test_dir="10_food_classes_10_percent/test/"

train_datagen= ImageDataGenerator(rescale=1/255.)

test_datagen= ImageDataGenerator(rescale=1/255.)

train_data10= train_datagen.flow_from_directory(train_dir,
                                                target_size=IMAGE_SHAPE,
                                                batch_size=BATCH_SIZE,
                                                class_mode="categorical")

test_data= test_datagen.flow_from_directory(test_dir,
                                           target_size=IMAGE_SHAPE,
                                           batch_size=BATCH_SIZE,
                                           class_mode="categorical")

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [ ]:
import datetime
import tensorflow as tf

def create_tensorboard_callbacks(dir_name, experiment_name):

  log_dir =dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%d%m%y-%H%M%S")
  tensorboard_callbacks =tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving Tensorboard log files to: {log_dir}")
  return tensorboard_callbacks

In [ ]:
resnet_url= "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

efficientnet_url=  "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"




In [ ]:
import tensorflow as tf
import tensorflow_hub as hub



In [ ]:
from tensorflow.keras import layers

In [ ]:

def create_model(model_url,num_classes=[10]):
  feature_extraction_layer= hub.KerasLayer(model_url,
                                         trainable=False,
                                         name="feature_extraction_layer",
                                         input_shape=IMAGE_SHAPE+ (3,))
  
  model= tf.keras.Sequential([
      feature_extraction_layer,
      layers.Dense(num_classes,activation="softmax", name="Output_layer")
  ])

  return model

In [ ]:
resnet_model = create_model(resnet_url,
                            num_classes=train_data10.num_classes)

In [ ]:
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer= tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [ ]:
resnet_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (K  (None, 2048)             23564800  
 erasLayer)                                                      
                                                                 
 Output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
resnet_model.fit(train_data10,
                 epochs=5,
                 steps_per_epoch=len(train_data10),
                 validation_data= test_data,
                 validation_steps= len(test_data),
                 callbacks=create_tensorboard_callbacks(dir_name="/content/Untitled Folder",
                                                        experiment_name="resnet test 01"))

Saving Tensorboard log files to: /content/Untitled Folder/resnet test 01/120623-103544
Epoch 1/5
24/24 [==============================] - 20s 815ms/step - loss: 0.4222 - accuracy: 0.8987 - val_loss: 0.6899 - val_accuracy: 0.7752
Epoch 2/5
24/24 [==============================] - 18s 785ms/step - loss: 0.3524 - accuracy: 0.9187 - val_loss: 0.6769 - val_accuracy: 0.7748
Epoch 3/5
24/24 [==============================] - 19s 806ms/step - loss: 0.2837 - accuracy: 0.9453 - val_loss: 0.6478 - val_accuracy: 0.7848
Epoch 4/5
24/24 [==============================] - 18s 779ms/step - loss: 0.2451 - accuracy: 0.9587 - val_loss: 0.6457 - val_accuracy: 0.7812
Epoch 5/5
24/24 [==============================] - 18s 772ms/step - loss: 0.2069 - accuracy: 0.9720 - val_loss: 0.6254 - val_accuracy: 0.7896
